![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/master/assets/img/ODPi_Egeria_Logo_color.png)

### Egeria Hands-On Lab
# Welcome to the Managing Servers Lab

## Introduction

Egeria is an open source project that provides open standards and implementation libraries to connect tools,
catalogues and platforms together so they can share information about data and technology (called metadata).

The Egeria Open Metadata and Governance (OMAG) Server Platform provides APIs for starting and stopping servers on a specific OMAG Server Platform.  This hands-on lab explains how this is done.

## The scenario

Gary Geeke is the IT Infrastructure leader at Coco Pharmaceuticals.  He has set up a number of OMAG Server Platforms and has validated that they are operating correctly.

![Gary Geeke](https://raw.githubusercontent.com/odpi/data-governance/master/docs/coco-pharmaceuticals/personas/gary-geeke.png)

In this hands-on lab Gary is starting and stopping servers on Coco Pharmaceutical's OMAG Server Platform.  Gary's userId is `garygeeke`.

In [ ]:
%run ../common/globals.ipynb

import requests

adminUserId     = "garygeeke"

In the [Egeria Server Configuration](../egeria-server-config.ipynb) lab, Gary configured servers for the OMAG Server Platforms shown in Figure 1:

![Figure 1](../images/coco-pharmaceuticals-systems-omag-server-platforms.png)
> **Figure 1:** Coco Pharmaceuticals' OMAG Server Platforms

Below are the host name and port number where the core, data lake and development platforms will run. 

In [ ]:
import os

corePlatformURL     = os.environ.get('corePlatformURL','https://localhost:9443') 
dataLakePlatformURL = os.environ.get('dataLakePlatformURL','https://localhost:9444') 
devPlatformURL      = os.environ.get('devPlatformURL','https://localhost:9445')

The commands to start and stop servers are part of the OMAG Server Platform's Operational Services which is a sub-component of the Administration Services.

The REST API calls all begin with the OMAG Server Platform URL followed by the following URL fragment

In [ ]:
operationalServicesURLcore = "/open-metadata/admin-services/users/" + adminUserId

## Exercise 1 - Starting a server on an OMAG Server Platform

A server is started by creating an instance of the server on the platform.  The command below starts `cocoMDS1` on the Data Lake OMAG Server Platform.

In [ ]:
import pprint
import json

serverName = "cocoMDS1"
platformURLroot = dataLakePlatformURL

print (" ")
print ("Starting server " + serverName + " ...")
url = platformURLroot + operationalServicesURLcore + '/servers/' + serverName + "/instance"
print ("POST " + url)

response = requests.post(url)

prettyResponse = json.dumps(response.json(), indent=4)
print ("Response: ")
print (prettyResponse)
print (" ")

----
The result shows all of the services that have been activated in the server.

If you check the command window where the OMAG Server Platform is running, you can see the console messages that record the initialization of the services requested in cocoMDS1’s configuration document.

Running this command again will restart the server.

----

## Exercise 2 - Querying the configuration of a running server

The services that are started within the server are defined in a configuration document.
As a reminder, the call to retrieve the configuration document for a particular server is as follows:

In [ ]:

serverName = "cocoMDS1"
platformURLroot = dataLakePlatformURL

print (" ")
print ("Retrieving stored configuration document for " + serverName + " ...")
url = platformURLroot + operationalServicesURLcore + '/servers/' + serverName + '/configuration'
print ("GET " + url)

response = requests.get(url)

if response.status_code == 200:
    relatedHTTPCode = response.json().get('relatedHTTPCode')
    if relatedHTTPCode == 200:
        print("Server configuration for " + serverName + " has been retrieved")
    else:
        print("Server configuration for " + serverName + " is unavailable")
else:
    print("Server Platform " + platformURLroot + " is unavailable")

serverConfig=response.json().get('omagserverConfig')


----

The [Understanding Server Configuration Lab](understanding-server-config.ipynb) walks through the content of the
configuration document.
This includes the audit trail that gives a high level overview of how the server has been configured.
This is a useful summary of a server's configuration.

---

In [ ]:

def getAuditTrail(sc):
    auditTrail=sc.get('auditTrail')
    if auditTrail == None:
        print ("Empty configuration - no audit trail - configure the server before continuing")
    else:    
        print ("Audit Trail: ")
        for x in range(len(auditTrail)): 
            print (auditTrail[x])

getAuditTrail(serverConfig)

----
However it is possible that the configuration document has been changed since the server was started.  This new configuration will not be picked up until the server restarts.

The following call retrieves the configuration that a running server is actually using so you can verify it is using the latest configuration.  Comparing the audit trail at the end of the running configuration with that of the configuration document is a
quick way to check if it has been changed.

----

In [ ]:

print (" ")
print ("Retrieving running configuration document for " + serverName + " ...")
url = platformURLroot + operationalServicesURLcore + '/servers/' + serverName + '/instance/configuration'
print ("GET " + url)

response = requests.get(url)

if response.status_code == 200:
    relatedHTTPCode = response.json().get('relatedHTTPCode')
    if relatedHTTPCode == 200:
        print("Server configuration for " + serverName + " has been retrieved")
        runningServerConfig=response.json().get('omagserverConfig')
        getAuditTrail(runningServerConfig)
    else:
        print("Server configuration for " + serverName + " is unavailable")
else:
    print("Server Platform " + platformURLroot + " is unavailable")


## Exercise 3 - shutting down a server

The command to shutdown a server is as follows:

In [ ]:
serverName = "cocoMDS1"
platformURLroot = dataLakePlatformURL

print (" ")
print ("Stopping server " + serverName + " ...")
url = platformURLroot + operationalServicesURLcore + '/servers/' + serverName + "/instance"
print ("DELETE " + url)

response = requests.delete(url)

prettyResponse = json.dumps(response.json(), indent=4)
print ("Response: ")
print (prettyResponse)
print (" ")

----
The command above is a temporary shutdown.

The following command is more permanent and should only be used if the server is not connecting to its
cohorts again.  Specifically, it shuts the server, unregisters it from the cohort and deleted the configuration document.

**Use this command with care :).**

----

In [ ]:
serverName = "myOldServer"
platformURLroot = dataLakePlatformURL

print (" ")
print ("Stopping server " + serverName + " ...")
url = platformURLroot + operationalServicesURLcore + '/servers/' + serverName
print ("DELETE " + url)

response = requests.delete(url)

prettyResponse = json.dumps(response.json(), indent=4)
print ("Response: ")
print (prettyResponse)
print (" ")

----

## What next?

* [Understanding Platform Services Lab](understanding-platform-services.ipynb) shows more commands for understanding the status of a running server.
* [Understanding Cohorts Lab](understanding-cohorts.ipynb) shows the commands for querying the status of the open metadata repository cohorts.

----